In [1]:
import pandas as pd
import json
import numpy as np


# reading in data
data = pd.read_csv('data/booksummaries.txt', sep='\t', header = None)

data.drop(columns=[0, 1], inplace=True)
data[2] = data[2].astype(str)

In [2]:
data.head()

,2,3,4,5,6
0,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [3]:
data[5][0]

'{"/m/016lj8": "Roman \\u00e0 clef", "/m/06nbt": "Satire", "/m/0dwly": "Children\'s literature", "/m/014dfn": "Speculative fiction", "/m/02xlf": "Fiction"}'

In [4]:


data.dtypes

2    object
3    object
4    object
5    object
6    object
dtype: object

In [5]:
# so this works now needs to be done to whole column 

# stdf = data[5].apply(json.loads)
# stdf = df['stats'].apply(json.loads)
# y = json.loads(data[5][1])

# test = data[5][0]

# type(test)
# y
# test

# droping any without tags

data[5].isnull().values.any()

data_2 = data.dropna(subset=[5])

data_2


,2,3,4,5,6
0,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
4,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...
5,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul Bäumer, a Ge..."
...,...,...,...,...,...
16549,The Third Lynx,Timothy Zahn,2007,"{""/m/06n90"": ""Science Fiction""}",The story starts with former government agent...
16551,Remote Control,Andy McNab,1997,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction...",The series follows the character of Nick Ston...
16555,Transfer of Power,Vince Flynn,2000-06-01,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...
16556,Decoded,Jay-Z,2010-11-16,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...


In [51]:
# sp here I toke apart the json

stdf = data_2[5].apply(json.loads)
stdf[0].values()


stdf = stdf.apply(lambda x: x.values())

stdf = stdf.apply(lambda x: list(x))

stdf = stdf.map(lambda x: x[0])

stdf

0                Roman à clef
1             Science Fiction
2              Existentialism
4        Hard science fiction
5                   War novel
                 ...         
16549         Science Fiction
16551                Thriller
16555                Thriller
16556           Autobiography
16558        Epistolary novel
Name: 5, Length: 12841, dtype: object

In [52]:

count_subj = stdf.explode().value_counts()

count_subj
# list(stdf[0])
# np.array(stdf[0])
# data_2.insert(0, "subjects", stdf)

# stdf_2 = stdf.apply(lambda x: x[0])

# stdf_3 = stdf_2.unique()

# len(stdf_3)


count_subj = count_subj.to_frame()
count_subj

,5
Science Fiction,2551
Speculative fiction,1438
Children's literature,1152
Fiction,943
Novel,925
...,...
Collage,1
Metaphysics,1
Colonial United States romance,1
Experimental literature,1


In [53]:
# so can i use these mystery, crime fiction, thriller, spy  all into one, fantasy with science fiction, young adult and childrens 

count_subj.loc[count_subj[5] > 100]

,5
Science Fiction,2551
Speculative fiction,1438
Children's literature,1152
Fiction,943
Novel,925
Mystery,734
Crime Fiction,672
Fantasy,624
Thriller,568
Young adult literature,321


In [47]:
# data_2.insert(0, "subjects", stdf)

# test = data_2['subjects'][0]

# type(test)

stdf

stdf = stdf.map(lambda x : 'Mystery' if(x == 'Crime Fiction') else x)
#  square = lambda x : x*x if(x > 0)

In [48]:
# data_2["subjects"][0][0]
stdf

0                Roman à clef
1             Science Fiction
2              Existentialism
4        Hard science fiction
5                   War novel
                 ...         
16549         Science Fiction
16551                Thriller
16555                Thriller
16556           Autobiography
16558        Epistolary novel
Name: 5, Length: 12841, dtype: object